In [1]:
import numpy as np
import pickle
import os
from scipy.sparse import csr_matrix
class Args:
	data = 'amazon'
	percent = 0
	graphSampleN = 15000
	batch = 512
	graphNum = 5
	keepRate = 0.5
	testSize = 1000
	test = True
	pos_length = 200
	sslNum = 80
args = Args()
class DataHandler:
	def __init__(self):
		if args.data == 'yelp':
			predir = './Datasets/Yelp/'
		elif args.data == 'gowalla':
			predir = './Datasets/gowalla/'
		elif args.data == 'amazon':
			predir = './Datasets/amazon/'
		else:
			predir='./Datasets/'+args.data+'/'
		predir = predir
		self.trnfile = predir + 'trn_mat_time'
		self.tstfile = predir + 'tst_int'
		self.sequencefile=predir+'sequence'
		self.test_dictfile=predir+'test_dict'
	def LoadData(self):
		if args.percent > 1e-8:
			print('noised')
			with open(self.predir + 'noise_%.2f' % args.percent, 'rb') as fs:
				trnMat = pickle.load(fs)
		else:
			with open(self.trnfile, 'rb') as fs:
				# print(pickle.load(fs))
				trnMat = pickle.load(fs)# (pickle.load(fs) != 0).astype(np.float32)
		# test set
		with open(self.tstfile, 'rb') as fs:
			tstInt = np.array(pickle.load(fs))
		with open(self.sequencefile, 'rb') as fs:
			self.sequence = pickle.load(fs)
		if os.path.isfile(self.test_dictfile):
			with open(self.test_dictfile, 'rb') as fs:
				self.test_dict = pickle.load(fs)
		# print("tstInt",tstInt)
		tstStat = (tstInt != None)
		# print("tstStat",tstStat,len(tstStat))
		tstUsrs = np.reshape(np.argwhere(tstStat != False), [-1])
		# print("tstUsrs",tstUsrs,len(tstUsrs))
		# self.trnMat = trnMat[0]
		def generate_rating_matrix_test(user_seq, num_users, num_items):
			# three lists are used to construct sparse matrix
			row = []
			col = []
			data = []
			for user_id, item_list in enumerate(user_seq):
				for item in item_list:  #
					row.append(user_id)
					col.append(item)
					data.append(1)

			row = np.array(row)
			col = np.array(col)
			data = np.array(data)
			rating_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_items))

			return rating_matrix
		args.user, args.item = trnMat[0].shape
		self.trnMat=generate_rating_matrix_test(self.sequence,args.user, args.item)
		self.subMat = trnMat[1]
		self.timeMat = trnMat[2]
		# print("trnMat",trnMat[0],trnMat[1],trnMat[2])
		self.tstInt = tstInt
		self.tstUsrs = tstUsrs
		self.prepareGlobalData()

	def prepareGlobalData(self):
		# adj = self.subMat
		self.maxTime=1
		# self.subMat,self.maxTime=self.timeProcess(self.subMat)
		# print(self.subMat[0],self.subMat[-1])

		self.item_with_pop=[]
	
handler = DataHandler()
handler.LoadData()

C:\Users\ssit5\AppData\Local\Temp\ipykernel_4132\2871647109.py:40: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  trnMat = pickle.load(fs)# (pickle.load(fs) != 0).astype(np.float32)


In [2]:
from collections import Counter
def filter_by_popularity(popular_percentage):
    item_interactions = Counter()
    user_interactions = Counter()
    for user_id, item_list in enumerate(handler.sequence):
        if user_id not in handler.tstUsrs:
            continue
        item_interactions.update(item_list)
        user_interactions[user_id] += len(item_list)
    items_cutoff = int(len(item_interactions) * popular_percentage)
    users_cutoff = int(len(user_interactions) * popular_percentage)
    sorted_items = sorted(item_interactions.items(), key=lambda x: x[1], reverse=True)
    sorted_users = sorted(user_interactions.items(), key=lambda x: x[1], reverse=True)
    popular_items = sorted([item for item, _ in sorted_items[:items_cutoff]])
    unpopular_items = sorted([item for item, _ in sorted_items[items_cutoff:]])
    popular_users = sorted([user for user, _ in sorted_users[:users_cutoff]])
    unpopular_users = sorted([user for user, _ in sorted_users[users_cutoff:]])
    # Convert to numpy arrays
    popular_items = np.array(popular_items, dtype=np.int32)
    unpopular_items = np.array(unpopular_items, dtype=np.int32)
    popular_users = np.array(popular_users, dtype=np.int32)
    unpopular_users = np.array(unpopular_users, dtype=np.int32)
    return popular_items, unpopular_items, popular_users, unpopular_users

In [3]:
# len(handler.sequence)
# handler.sequence[0]
# sorted_ids = np.argsort(handler.tstInt)
# sorted_ids
# for x in handler.tstInt:
#     if x is not None:
#         print(x)
len(handler.subMat)

5

In [ ]:
def sampleTestBatch(batIds, labelMat): # labelMat=TrainMat(adj)
    batch = len(batIds)
    temTst = handler.tstInt[batIds]
    temLabel = labelMat[batIds].toarray()
    temlen = batch * args.testSize# args.item
    uLocs = [None] * temlen
    iLocs = [None] * temlen
    uLocs_seq = [None] * temlen
    tstLocs = [None] * batch
    sequence = [None] * args.batch
    mask = [None]*args.batch
    cur = 0
    val_list=[None]*args.batch
    for i in range(batch):
        if(args.test==True):
            posloc = temTst[i]
        else:
            posloc = handler.sequence[batIds[i]][-1]
            val_list[i]=posloc
        rdnNegSet = np.array(handler.test_dict[batIds[i]+1][:args.testSize-1])-1
        locset = np.concatenate((rdnNegSet, np.array([posloc])))
        tstLocs[i] = locset
        for j in range(len(locset)):
            uLocs[cur] = batIds[i]
            iLocs[cur] = locset[j]
            uLocs_seq[cur] = i
            cur += 1
        sequence[i]=np.zeros(args.pos_length,dtype=int)
        mask[i]=np.zeros(args.pos_length)
        if(args.test==True):
            posset=handler.sequence[batIds[i]]
        else:
            posset=handler.sequence[batIds[i]][:-1]
        # posset=handler.sequence[batIds[i]]
        if(len(posset)<=args.pos_length):
            sequence[i][-len(posset):]=posset
            mask[i][-len(posset):]=1
        else:
            sequence[i]=posset[-args.pos_length:]
            mask[i]+=1
    if(batch<args.batch):
        for i in range(batch,args.batch):
            sequence[i]=np.zeros(args.pos_length,dtype=int)
            mask[i]=np.zeros(args.pos_length)
    return uLocs, iLocs, temTst, tstLocs, sequence, mask, uLocs_seq, val_list

def sampleSslBatch(batIds, labelMat, use_epsilon=True):
    temLabel=list()
    for k in range(args.graphNum):
        temLabel.append(labelMat[k][batIds].toarray())
    batch = len(batIds)
    temlen = batch * 2 * args.sslNum
    uLocs = [[None] * temlen] * args.graphNum
    iLocs = [[None] * temlen] * args.graphNum
    uLocs_seq = [[None] * temlen] * args.graphNum
    # epsilon=[[None] * temlen] * args.graphNum
    for k in range(args.graphNum):	
        cur = 0				
        for i in range(batch):
            posset = np.reshape(np.argwhere(temLabel[k][i]!=0), [-1])
            # print(posset)
            sslNum = min(args.sslNum, len(posset)//2)# len(posset)//4# 
            if sslNum == 0:
                poslocs = [np.random.choice(args.item)]
                neglocs = [poslocs[0]]
            else:
                all = np.random.choice(posset, sslNum*2) #- args.user
                # print(all)
                poslocs = all[:sslNum]
                neglocs = all[sslNum:]
            for j in range(sslNum):
                posloc = poslocs[j]
                negloc = neglocs[j]			
                uLocs[k][cur] = uLocs[k][cur+1] = batIds[i]
                uLocs_seq[k][cur] = uLocs_seq[k][cur+1] = i
                iLocs[k][cur] = posloc
                iLocs[k][cur+1] = negloc
                cur += 2
        uLocs[k]=uLocs[k][:cur]
        iLocs[k]=iLocs[k][:cur]
        uLocs_seq[k]=uLocs_seq[k][:cur]
    return uLocs, iLocs, uLocs_seq

def testEpoch(popularity_users=None, popularity_items=None):
    def generate_rating_matrix_test(user_seq, num_users, num_items, items_to_keep=None):
        # three lists are used to construct sparse matrix
        row = []
        col = []
        data = []
        included_users = set()
        for user_id, item_list in enumerate(user_seq):
            for item in item_list:
                if items_to_keep is not None and item not in items_to_keep:
                    continue
                row.append(user_id)
                included_users.add(user_id)
                col.append(item)
                data.append(1)
        row = np.array(row)
        col = np.array(col)
        data = np.array(data)
        rating_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_items))
        return rating_matrix, included_users
    pi, ui, pu, uu = filter_by_popularity(0.1)
    
    ids = handler.tstUsrs
    if popularity_users == True:
        ids = pu
    elif popularity_users == False:
        ids = uu

    trnMat = handler.trnMat
    if popularity_items == True:
        trnMat, included_users = generate_rating_matrix_test(handler.sequence, args.user, args.item, items_to_keep=pi)
        induced_ids = []
        for user_id in ids:
            if user_id not in included_users:
                continue
            induced_ids.append(user_id)
        ids = np.array(induced_ids, dtype=np.int32)
    elif popularity_items == False:
        trnMat, included_users = generate_rating_matrix_test(handler.sequence, args.user, args.item, items_to_keep=ui)
        induced_ids = []
        for user_id in ids:
            if user_id not in included_users:
                continue
            induced_ids.append(user_id)
        ids = np.array(induced_ids, dtype=np.int32)

    num = len(ids)
    tstBat = args.batch
    steps = int(np.ceil(num / tstBat))
    # np.random.seed(100)
    for i in range(steps):
        st = i * tstBat
        ed = min((i+1) * tstBat, num)
        batIds = ids[st: ed]
        feed_dict = {}
        # print("batIds",batIds)
        uLocs, iLocs, temTst, tstLocs, sequence, mask, uLocs_seq, val_list = sampleTestBatch(batIds, trnMat)
        suLocs, siLocs, _ = sampleSslBatch(batIds, handler.subMat, False)
        feed_dict["uids"] = uLocs
        feed_dict["iids"] = iLocs
        feed_dict["sequence"] = sequence
        feed_dict["mask"] = mask
        feed_dict["uLocs_seq"] = uLocs_seq
        # print("test",uLocs_seq)
        for k in range(args.graphNum):
            feed_dict[f"suids[{k}]"] = suLocs[k]
            feed_dict[f"siids[{k}]"] = siLocs[k]
        print(handler.subMat)
        break
    # print(feed_dict)

testEpoch(popularity_users=True)
testEpoch(popularity_users=False)

[<11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 72280 stored elements in Compressed Sparse Row format>, <11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 78997 stored elements in Compressed Sparse Row format>, <11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 79692 stored elements in Compressed Sparse Row format>, <11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 78096 stored elements in Compressed Sparse Row format>, <11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 45651 stored elements in Compressed Sparse Row format>]
[<11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 72280 stored elements in Compressed Sparse Row format>, <11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 78997 stored elements in Compressed Sparse Row format>, <11199x30821 sparse matrix of type '<class 'numpy.int32'>'
	with 79692 stored elements in Compressed Sparse Row format>, <11199x30821 sparse matrix of 